# Algorithm Implementation

by [Lei You](http://user.it.uu.se/~leiyo378)

In this document, we solve the flexible TTI allocation problem to optimum, by using our proposed algorithm. The formulation of the original problem is as below.

$$
\begin{align}
[\text{P0}] \quad\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} \\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1 \quad i\in\mathcal{I} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$

We have $x_{b,k}=1$ if and only if physical resource block (PRB) $b$ is allocated to service $k$.

Recall that $\tau_k$ is the maximum tolerant latency of service $k$, and $d_b$ is the end time of the PRB $b$. The constraints for the latency is imposed, by letting $r_{b,k}$ follow the rule:
$$
r_{b,k}=\left\{
\begin{array}{ll}
0 & \text{if } \tau_k-d_b<0 \\
\text{capacity} & \text{otherwise}
\end{array}
\right.
$$

Here we go.

In [14]:
import sys
import scipy.io
import numpy
import math
import csv
from operator import add

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

epsilon = 10e-6

The sets $\mathcal{B}$, $\mathcal{K}^{(\ell)}$, $\mathcal{K}^{(c)}$, $\mathcal{K}$, and $\mathcal{I}$ are read by the following code.

In [15]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B = list(B_csv)
    B = [item for sublist in B for item in sublist] # flatten list
    B = map(int, map(float, B)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

The parameters $\mathbf{r}$, $\mathbf{q}$, and $\mathbf{a}$ are read below.

In [16]:
# # matrix r
with open('r.csv', 'rb') as f:
    r_csv = csv.reader(f)
    r = list(r_csv)
    r = [ map(int,map(float,x)) for x in r] # convert to int
    r = numpy.matrix(r)

# # vector q, only for Kl
with open('q.csv', 'rb') as f:
    q_csv = csv.reader(f)
    q = list(q_csv)
    q = [item for sublist in q for item in sublist] # flatten list
    q = map(int, map(float, q)) # convert to int
    q = numpy.array(q)
    
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

The following code reads the overlapping relationship between any two PRBs.

In [17]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
with open('conflict_PRB.csv', 'rb') as f:
    conflict_PRB_csv = csv.reader(f)
    conflict_PRB = list(conflict_PRB_csv)
    conflict_PRB = [ map(lambda y: int(float(y))==1, x) for x in conflict_PRB]

In [18]:
with open('lp_x.csv', 'rb') as f:
    lp_x_csv = csv.reader(f)
    lp_x = list(lp_x_csv)
    lp_x = [ map(float, t) for t in lp_x] # convert to float

The original problem is relaxed by Lagrangian with $\mathbf{\lambda}>\mathbf{0}$:

The Lagrangian is as follows.

$$
\begin{align}
[\text{P1}]\quad g(\mathbf{\lambda})=\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} + \sum_{i\in\mathcal{I}}\lambda_i(1-\sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k})\\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$

For any fixed $\mathbf{\lambda}$, the orginal problem decomposes to two problems in respect of $\mathcal{K}^{(c)}$ and $\mathcal{K}^{(\ell)}$. For the sake of presentation, we denote 
$$\alpha_b = \sum_{i\in\mathcal{I}}\lambda_i a_{b,i}$$ 

In [19]:
def getAlpha(lam):
    alpha=[]
    for b in B:
        alpha_b = sum( lam[i]*a[b,i] for i in I ) 
        alpha.append(alpha_b)
    return alpha

For $\mathcal{K}^{(c)}$, the problem is as follows.
$$
\begin{align}
[\text{P2}]\quad\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} - \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}}\alpha_{b}x_{b,k}\\
s.t. \quad & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}^{(c)}
\end{align}
$$

In the above formulation, it might happen that one PRB is allocated to multiple services simultaneously, which definitely leads to PRB overlap. Therefore we add an extra constraint such that the problem becomes:
$$
\begin{align}
[\text{P2}]\quad\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}}x_{b,k}( r_{b,k} - \alpha_{b})\\
s.t. \quad & \sum_{k\in\mathcal{K}^{(c)}}x_{b,k}\leq 1 \quad b\in\mathcal{B} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}^{(c)}
\end{align}
$$

It can be optimally solved by:
For each $b\in\mathcal{B}$, we select one $k$, such that.
$
r_{b,k}-\alpha_b > 0 \text{ and } k = \arg\max_{k\in\mathcal{K}^{(c)}} r_{b,k}-\alpha_b
$.
The corresponding code is as follows.

In [20]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kc being computed.
# The columns in Kl of the returned matrix x are zero vectors.
def solveP2( lam ):
    PRB_alloc = [ -1 for b in B ] # PRB_alloc[b] is the index of service that PRB b should be allocated to
                                  # PRB_alloc[b]=-1 means that PRB b is not allocated
        
    sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
    
    alpha = getAlpha(lam)
    for b in B:
        tmp_list = [ r[b,k]-alpha[b] for k in Kc ]
        if max(tmp_list) > 0:   # PRB is allocated only if r[b][k]-alpha[b] is positive
            PRB_alloc[b] = len(Kl) + numpy.argmax(tmp_list) # Kl is added such that the value of PRB_alloc[b] is 
                                                       # coherent with the corresponding indexed position in K
    
    # Convert PRB_alloc to matrix x
    for b in B:
        k = PRB_alloc[b] # indexed service
        if k >= 0: # indicating that PRB_alloc[b] != -1
            sol_x[b][k] = 1
    
    return sol_x

For $\mathcal{K}^{(\ell)}$, we have the problem below.
$$
\begin{align}
[\text{P3}]\quad\min_{\mathbf{x}} \quad &  \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(\ell)}}\alpha_{b}x_{b,k}\\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}^{(\ell)}
\end{align}
$$

The problem can decomposed to $|\mathcal{K}^{(\ell)}|$ knapsack problems and be optimally solved by dynamic programming. 

For $k\in\mathcal{K}^{(\ell)}$:
$$
\begin{align}
[\text{P3[$k$]}]\quad\min_{\mathbf{x}} \quad &  \sum_{b\in\mathcal{B}}\alpha_{b}x_{b,k}\\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k}  \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B}
\end{align}
$$

Though the multiple knapsack problem can still be exactly solved by dynamic programming, here we use gurobi integer programming solver instead, without loss of optimality.

In [21]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kl being computed.
# The columns in Kc of the returned matrix x are zero vectors.
def solveP3( lam ):
    # create optimization model
    modelKl = Model('Integer Programming - Kl')
    modelKl.modelSense = GRB.MINIMIZE
    modelKl.setParam('OutputFlag', False) # slience output
    
    # create varialbes for modelKl:
    xP3 = []
    for b in B:
        xP3_b = []
        for k in Kl:
            xP3_b.append(modelKl.addVar(vtype=GRB.BINARY))
        xP3.append(xP3_b)
    modelKl.update()
    
    # add constraints 
    for k in Kl:
        modelKl.addConstr( sum(r[b,k]*xP3[b][k] for b in B) >= q[k] )
    modelKl.update()

    # set objective function
    alpha = getAlpha(lam)
    modelKl.setObjective(
        sum( alpha[b]*xP3[b][k] for k in Kl for b in B )
    )
    
    # solve modelKl
    modelKl.optimize()
    
    # construct variables to be returned 
    sol_x = [ [ 0 for k in K ] for b in B ]
    for b in B:
        for k in Kl:
            sol_x[b][k] = int(xP3[b][k].x)
            
    return sol_x

The solutions obtained by $\texttt{solveP2}$ and $\texttt{solveP3}$ can be merged to obtain the value of Lagrangian Dual function, by:

In [22]:
# This function evaluates the Lagrangian Dual function for any given vector lamda
# The return is the matrix x in the original problem
def solveG(lam):
    return ( numpy.matrix(solveP2(lam)) + numpy.matrix(solveP3(lam)) ).tolist()
    # The  return value is optimal to the maximization in lagrangian dual function 
    # but may not be feasible to the original problem!

Next, we use subgradient descent method to solve the Lagrangian Dual problem, i.e., $\min_{\mathbf{\lambda}\geq\mathbf{0}} g(\mathbf{\lambda})$. After the completion of the gradient descent method, the obtained solution is not guaranteed to be feasible to the primal problem. The heuristic method to obtain a feasible solution is as follows: All PRBs allocation for $\mathcal{K}^{(\ell)}$ is kept so as to guarantee the latency constraints being satisfied. Then we solve $\mathcal{K}^{(c)}$ under the current lambda and fixed $\mathcal{K}^{(\ell)}$ solutions.

To achieve this goal, we need to implement a new function that solves Kc with some PRB allocations being fixed:

In [23]:
def getHeuristicX(lam, x_count):
    priority = [ [i[0] for i in sorted(enumerate(x_count[k]), key=lambda y:y[1],reverse=True)] for k in K ]
    sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
    
    priority_user = [i[0] for i in sorted(enumerate(map(sum,x_count)), key=lambda y:y[1],reverse=True)] 
    priority_Kl = [u for u in priority_user if u in Kl ]
    priority_Kc = [u for u in priority_user if u in Kc ]
    
    collision = [ False for b in B ]
    
    for k in priority_Kl:
        service_bit = sum(r[b,k]*sol_x[b][k] for b in B)
        while service_bit < q[k] and collision.count(False)>0: 
            for pos in range(len(priority[k])):
                b = priority[k][pos]
                if collision[b] == False: 
                    sol_x[b][k] = 1
                    service_bit += r[b,k]*sol_x[b][k]
                    for p in B: # set all PRB overlapping with b to be in collision
                        if conflict_PRB[b][p] == True:
                            collision[p] = True
                    break
    
    alpha = getAlpha(lam)
    for b in B:
        alloc = True
        if collision[b]==True: # b wouldn't be allocated if in collision
            alloc = False 
        if alloc == True: 
            tmp_list = [ r[b,k]-alpha[b] for k in Kc ]
            user_to_alloc = len(Kl) + numpy.argmax(tmp_list) 
            sol_x[b][user_to_alloc] = 1
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[b][p] == True:
                    collision[p] = True
                    
    return sol_x
        

The following function checks whether a solution is primal feasible.

In [24]:
def isFeasible(x):
    if (numpy.dot( numpy.dot(a.transpose(), 
                             numpy.matrix(x)), numpy.ones(len(K)) ) 
        > numpy.ones(len(I))).tolist()[0].count(True)>0:
        return False
    if (numpy.dot(numpy.multiply(r,numpy.matrix(x))[:,0:len(Kl)].transpose(),
                  numpy.ones(len(B)))<q).tolist()[0].count(True)>0:
        return False
    return True

The gradient descent along with the heuristic afterwards is implemented as follows.

In [25]:
lam = [ 1 for i in I ]
x_prev = [ [0 for k in K ] for b in B ]
penalty = [ 1 for i in I ]
x_count = [ [ 0 for b in B ] for k in K ]
best_dual_sofar = 1e10
eta = 0.95
step_para = default_step_para = 0.5
no_improve_count = 0

In [26]:
for it in range(1, 151): # k belongs to [1,100]
    
    xP3 = solveP3(lam)
    xP2 = solveP2(lam)
    x = (numpy.matrix(xP3) + numpy.matrix(xKc)).tolist() 
    
    
    # obtain the corresponding dual function value under current lambda
    
    dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
    if dual < best_dual_sofar + 0.005:
        no_improve_count = 0
        best_dual_sofar = dual
        step_para = min(step_para*1.05, 1.9)
    else:
        no_improve_count += 1
        
    if no_improve_count >=2:
        step_para = 0.95*step_para
        no_improve_count = 0
    
    gamma = abs(dual - 2107*0.9)
    gk = numpy.linalg.norm(penalty, 2)**2
    
    if it<5:
        step_len = 10/float(it**0.5)
    else:
        step_len = step_para*gamma/gk
    
    penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
    lam = map(lambda y:max(y,0), (numpy.matrix(lam) - step_len*numpy.matrix(penalty)).tolist()[0])

    x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()    
    
    print 'iteration', it, 'step length=', step_len, 'dual=', best_dual_sofar, 'step_para=', step_para

TypeError: list indices must be integers, not tuple

In [ ]:
best = 0
x_count2 = numpy.multiply(numpy.matrix(lp_x).transpose(), numpy.matrix(x_count)).tolist()
print 'calculating final solution ... '
for i in range(400):
    heuristic_x = getHeuristicX2(lam, x_count) 
    curr = sum(r[b,k]*heuristic_x[b][k] for k in Kc for b in B)
    if curr > best:
        best = curr
print 'feasibility=', isFeasible(heuristic_x), '\n best obj=', best

In [ ]:
# import pandas
# heuristic_x = getHeuristicX2(lam, x_count) 
# print 'allocated blocks number:', [ sum(numpy.matrix(heuristic_x).transpose()[k].tolist()[0]) for k in K ], 'obj=', sum(r[b][k]*heuristic_x[b][k] for k in Kc for b in B)
# indices = [ [ [i,k] for i,x in enumerate(numpy.matrix(heuristic_x)[:,k].transpose().tolist()[0]) if x==1 ] for k in K]
# pandas.DataFrame(indices)